In [19]:
import tqdm
import argparse
import os
import torch
import fire
import torch.nn.functional as F
from torchaudio.transforms import Resample
import librosa
import torch.nn as nn
import torch.nn.functional as F
import torchaudio.transforms as trans
import sys
from pathlib import Path

# Add the parent directory to sys.path
sys.path.append("/home/leying.zhang/code/seed-tts-eval/thirdparty/UniSpeech/downstreams/speaker_verification")
from verification_pair_list_v2 import * 

filename_dict = {}
wav_list = ["fe_03_06301_030.txt","fe_03_06310_011.txt","fe_03_06324_021.txt","fe_03_06333_011.txt","fe_03_06358_003.txt","fe_03_06380_016.txt","fe_03_06386_033.txt","fe_03_09707_021.txt","fe_03_09715_008.txt","fe_03_09715_029.txt","fe_03_09720_004.txt","fe_03_09724_023.txt","fe_03_09727_028.txt","fe_03_09729_006.txt","fe_03_09761_016.txt","fe_03_09769_011.txt","fe_03_09779_030.txt"]
for i in range(len(wav_list)):
    wav = wav_list[i]
    test_name = wav.split(".")[0]+".wav"
    if i >= 9: 
        test_index = "000000"+str(i+1)+".wav"
    else:
        test_index = "0000000"+str(i+1)+".wav"
    filename_dict[test_name] = test_index
    
print(filename_dict)

{'fe_03_06301_030.wav': '00000001.wav', 'fe_03_06310_011.wav': '00000002.wav', 'fe_03_06324_021.wav': '00000003.wav', 'fe_03_06333_011.wav': '00000004.wav', 'fe_03_06358_003.wav': '00000005.wav', 'fe_03_06380_016.wav': '00000006.wav', 'fe_03_06386_033.wav': '00000007.wav', 'fe_03_09707_021.wav': '00000008.wav', 'fe_03_09715_008.wav': '00000009.wav', 'fe_03_09715_029.wav': '00000010.wav', 'fe_03_09720_004.wav': '00000011.wav', 'fe_03_09724_023.wav': '00000012.wav', 'fe_03_09727_028.wav': '00000013.wav', 'fe_03_09729_006.wav': '00000014.wav', 'fe_03_09761_016.wav': '00000015.wav', 'fe_03_09769_011.wav': '00000016.wav', 'fe_03_09779_030.wav': '00000017.wav'}


In [20]:

test_name = "fe_03_06324_021.wav"
t1_path = "/home/leying.zhang/code/CoVoMix/prompt/"+test_name.replace(".wav","_2.wav") # prompt
t2_path = "/home/leying.zhang/code/CoVoMix/SpeechSimilarityTestDiag/voice1covomix/"+filename_dict[test_name] # covomix
t3_path = "/home/leying.zhang/code/CoVoMix/SpeechSimilarityTestDiag/voice2/"+filename_dict[test_name] # covosingle
# t2_path = "/exp/leying.zhang/CoVoMix/soundstorm_demo/soundstorm_8k/"+name+"_1.wav" # target

t1_path = "/exp/leying.zhang/Fisher-dataset-test/2spk_soundstorm_simu/ground_truth/0.wav"
t2_path = "/exp/leying.zhang/Fisher-dataset-test/2spk_soundstorm_simu/prompt_spk1/0.wav"

covomix_time = [0,1.22]

sr = 16000
model = None
sim1, model = verification_ly("wavlm_large", t1_path, t2_path, 
                          use_gpu=True, checkpoint="/exp/leying.zhang/pretrained_models/wavlm_large_finetune.pth", 
                          wav1_start_sr=0, 
                          wav1_end_sr=-1, 
                          wav2_start_sr=0,
                          wav2_end_sr=-1, 
                          model=model, 
                          wav2_cut_wav1=False, device="cuda",
                          cut_prompt = True)

# sim2, model = verification("wavlm_large", t1_path, t3_path, 
#                           use_gpu=True, checkpoint="/exp/leying.zhang/pretrained_models/wavlm_large_finetune.pth", 
#                           wav1_start_sr=0, 
#                           wav1_end_sr=-1, 
#                           wav2_start_sr=int(covosingle_time[0]*sr),
#                           wav2_end_sr=int(covosingle_time[1]*sr), 
#                           model=model, 
#                           wav2_cut_wav1=False, device="cuda")
print("sim of covomix: ",sim1,)

import librosa
import numpy as np
from soundfile import write
from scipy.io import wavfile

sr = 8000
def extract_segment(input_file,start_sample, end_sample, output_file):
    # Load the WAV file
    sample_rate, audio_data = wavfile.read(input_file)
    if end_sample != -1 and (start_sample < 0 or end_sample > len(audio_data) or start_sample >= end_sample):
        raise ValueError("Invalid start or end sample indices.")
    segment = audio_data[start_sample:end_sample]
    wavfile.write(output_file, sample_rate, segment)

extract_segment(t1_path, 0, int(covomix_time[1]*sr), "/home/leying.zhang/tmp/test-gt.wav")

NameError: name 'verification_ly' is not defined

In [1]:

a = [[-1,0,2,1,1,-2,-2,-3,-1,-1],
[0,-2,-1,-1,0,-3,1,2,-3,2],
[-1,0,2,-1,3,0,2,0,2,3],
[-1,1,3,2,2,2,1,-2,2,3],
[-1,-2,-2,-2,-1,2,2,3,-1,-2],
[0,0,0,0,1,-1,-1,0,0,-2],
[-1,-1,1,-1,-2,-1,2,0,0,-2],
[0,0,1,-2,1,-2,1,2,-2,3],
[-1,-2,-2,-3,0,0,-2,-1,-2,-2],
[-1,-2,1,-3,0,-3,-2,0,-2,-3]]


s = 0
l = [2,3,5,6,7]
for i in range(len(a)):
    for j in a[i]:
        # print(j)
        if i in l:
            # print(j)
            s = s-j
        else: 
            s = s+j
print(s, s/50)

-62 -1.24
